In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import logging
from datetime import datetime, timezone
from typing import List

from redis.asyncio import Redis

from app.database_redis.connection import get_redis_client
from app.database_redis.dals.connection_dal import ConnectionDAL
from app.services.apis.streamqueue_service.client import StreamQueueServiceAPI
from app.services.apis.streamqueue_service.schemas import AudioChunkInfo, ExistingConnectionInfo
from app.services.audio.redis import Connection, Diarizer, Meeting, Transcriber
from app.settings import settings

from app.services.audio.audio import AudioSlicer


check_and_process_connections_interval_sec: 1.0


In [4]:
from app.services.audio.redis import Diarisation, Diarizer, Meeting, best_covering_connection,connection_with_minimal_start_greater_than_target


In [5]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [6]:
meeting = Meeting(redis_client,'test_meeting_id')
await meeting.load_from_redis()

In [7]:
connections = await meeting.get_connections()

In [8]:
len(connections)

16

### test on corrupted file 

#### 1 which connection is goin to be chosen - copy and corrupt that file 

In [9]:
current_time = datetime.now(timezone.utc)

In [10]:
current_time

datetime.datetime(2024, 6, 4, 18, 38, 13, 823128, tzinfo=datetime.timezone.utc)

In [11]:
meeting.diarizer_seek_timestamp

datetime.datetime(2024, 6, 4, 11, 35, 57, 670813, tzinfo=tzutc())

In [23]:
connection = best_covering_connection(meeting.diarizer_seek_timestamp, current_time, connections)

In [24]:
await connection.load_from_redis()

In [28]:
corrupt_connection = Connection(redis_client,'corrupt_connection_id')
await connection.update_timestamps(connection.start_timestamp, connection.end_timestamp)

In [29]:
seek = (meeting.diarizer_seek_timestamp - connection.start_timestamp).total_seconds()

In [32]:
path = f"/audio/{connection.id}.webm"
corrupt_path = f"/audio/{corrupt_connection.id}.webm"

In [33]:
corrupt_path

'/audio/corrupt_connection_id.webm'

In [34]:
with open(path,'rb') as f:
    data = f.read()
    corrupted_data = data[10:]
    
with open(corrupt_path,'wb') as f:
    f.write(corrupted_data)

In [35]:
audio_slicer = await AudioSlicer.from_ffmpeg_slice(corrupt_path, 0, 240)

2024-06-04 18:50:26,907 - ERROR - app.services.audio.audio - header is corrupted for audio file: /audio/corrupt_connection_id.webm


AudioFileCorruptedError: Audio File header /audio/corrupt_connection_id.webm is corrupted

#### 2 add this one to working diarizer 

In [37]:
meeting.meeting_id

'test_meeting_id'

In [43]:
diarizer = Diarizer(redis_client)
await diarizer.add_todo(meeting.meeting_id)

#### 3 

In [44]:
def connection_with_minimal_start_greater_than_target(target_start, connections):
    best_connection = None
    min_start_timestamp = None

    for connection in connections:
        if connection.start_timestamp > target_start:
            if min_start_timestamp is None or connection.start_timestamp < min_start_timestamp:
                min_start_timestamp = connection.start_timestamp
                best_connection = connection

    return best_connection

In [46]:
connection_with_minimal_start_greater_than_target(meeting.diarizer_seek_timestamp,connections)

In [45]:
connections